# Time Series Clustering Evaluation

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from math import ceil

import plotly.plotly as py
import plotly.offline as po
import plotly.graph_objs as go
import plotly.tools as tools
import colorlover as cl
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.go_offline()

import matplotlib. pyplot as plt
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap


from features.feature_ts import *
import evaluation.clusteval as ce
from observations.obs_processing import *
from support import data_dir

## Cluster Results

Fetch results from file.

In [ ]:
cluster_results = ce.readResults()
cluster_results.head()

In [ ]:
def plotClusterIndex(index, title, ylog=False):
    experiments = cluster_results.experiment_name.unique()
    colours = dict(zip(experiments, cl.scales[str(len(experiments))]['qual']['Paired']))
    df = cluster_results.set_index(['experiment_name','series'])[[index,'clusters']]
    data = pd.pivot_table(df[[index,'clusters']], index='clusters', columns=df.index, values=index)

    #generate plot data
    traces = []
    for c in data.columns:
        x = data.index
        y = data[c]
        n = '_'.join([c[0].split('_',1)[1],str(c[1])])
        hovertext = list()
        for i in x:
            hovertext.append('{}<br />{}: {:.3f}<br />{} clusters<br />'.format(n, index, y[i], i))

        traces.append(dict(
            x=x,
            y=y,
            name=n,
            legendgroup=c[0].split('_',1)[1],
            mode='lines+markers',
            marker=dict(size=3),
            line=dict(color=colours[c[0]]),
            text = hovertext,
            hoverinfo='text',
            connectgaps=True
        ))

    #set layout
    if ylog == True:
        yax = dict(title = index+' (log scale)' , type='log')
    else:
        yax = dict(title = index)
    layout = go.Layout(
            title= title,
            margin=go.Margin(t=50,r=50,b=50,l=50, pad=10),
            height= 700,
            xaxis=dict(title = 'clusters (log scale)', type='log'),
            yaxis=yax,
            hovermode = "closest"
            )

    fig = {'data':traces, 'layout':layout }
    return po.iplot(fig)

### Davies-Bouldin Index

In [ ]:
plotClusterIndex('dbi', 'Davies-Bouldin Index')

### Mean Index Adequacy

In [ ]:
plotClusterIndex('mia','Mean Index Adequacy')

### Silhouette Score

The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters. Negative values generally indicate that a sample has been assigned to the wrong cluster, as a different cluster is more similar.

In [ ]:
plotClusterIndex('silhouette', 'Silhouette Score')

### Selecting Best Clusters

In [ ]:
plotClusterIndex('score','Combined Cluster Score',ylog=True)

In [ ]:
demin_clusters = ce.selectClusters(cluster_results, 5, 'exp2_kmeans_demin')
zerone_clusters = ce.selectClusters(cluster_results, 5, 'exp2_kmeans_zero-one')
sanorm_clusters = ce.selectClusters(cluster_results, 5, 'exp2_kmeans_sa_norm')
best_clusters = ce.selectClusters(cluster_results, 10)

## Explore Best Clusters

In [ ]:
def plotClusterCentroids(best_clusters, n_best=1):
    
    best_experiments = list(best_clusters.experiment_name.unique())
    centroid_files = dict(zip(best_experiments,[e+'_centroids.csv' for e in best_experiments]))
    centroids = {}
    for k, v in centroid_files.items():
        centroids[k] = pd.read_csv(os.path.join(data_dir, 'cluster_results', v))
    
    best_centroids = pd.DataFrame()
    for row in best_clusters.itertuples():
        df = centroids[row.experiment_name]
        c = df.loc[(df.som_dim==row.som_dim)&(df.n_clust==row.n_clust),:]
        best_centroids = best_centroids.append(c)
    best_centroids.drop_duplicates(subset=['som_dim','n_clust','k','experiment_name'],keep='last',inplace=True)
    
    experiment_name, som_dim, n_clust = best_clusters.loc[n_best-1,['experiment_name','som_dim','n_clust']]    
    val = best_centroids['n_clust'].max()
    
    data = best_centroids.set_index(['experiment_name','som_dim','n_clust','k'])
    data.sort_index(level=['experiment_name','som_dim','n_clust'], inplace=True)
    traces = data.loc[(experiment_name, som_dim, n_clust), [str(i) for i in range(0,24)]].reset_index(drop=True).T
    traces.columns = ['cluster ' + str(k+1) for k in traces.columns.values]
    
    clust_size = data.loc[(experiment_name, som_dim, n_clust), 'cluster_size'].reset_index(drop=True)
    largest = 'cluster '+str(clust_size.idxmax()+1)
    
    colours = cl.interp(cl.scales['12']['qual']['Paired'], 100)[:n_clust]
    
    fig = tools.make_subplots(rows=3, cols=1, shared_xaxes=False, specs=[[{'rowspan': 2}],[None],[{}]],
                              subplot_titles=['cluster profiles '+experiment_name+' (n='+str(n_clust)+') TOP '+str(n_best),'cluster sizes'], print_grid=False)  
    i = 0
    for col in traces.columns:
        if col == largest:
            width = 3
        else:
            width = 1
        fig.append_trace({'x': traces.index, 'y': traces[col], 'line':{'color':colours[i],'width':width}, 'type': 'scatter', 'name': col}, 1, 1)
        i+=1
    fig.append_trace({'x': traces.columns, 'y': clust_size, 'type': 'bar', 'name': str(n_clust)+' clusters'} , 3, 1)
    
    fig['layout']['xaxis1'].update(title='time of day')
    fig['layout']['xaxis2'].update(tickangle=270)
    fig['layout']['yaxis1'].update(title='normalised load profile')
    fig['layout']['yaxis2'].update(title='profile count')
    fig['layout']['margin'].update(t=50,r=80,b=100,l=90,pad=10),
    fig['layout'].update(height=700, hovermode = "closest")
    
    po.iplot(fig)

## Analyse Cluster Label Patterns

In [ ]:
best_experiments = list(best_clusters.experiment_name.unique())
label_files = dict(zip(best_experiments,[e+'_labels.feather' for e in best_experiments]))
labels = {}
for k, v in label_files.items():
    labels[k] = feather.read_dataframe(os.path.join(data_dir, 'cluster_results', v))

In [ ]:
demin_labels = ce.bestLabels('exp2_kmeans_demin')
zerone_labels = ce.bestLabels('exp2_kmeans_zero-one')
sanorm_labels = ce.bestLabels('exp2_kmeans_sa_norm')
norm_labels = ce.bestLabels('exp2_norm_kmeans')

In [ ]:
demin_labels.head()

In [ ]:
def plotClusterLabels(label_data, year, n_clust=None, som_dim=0):
    
    if n_clust is None:
        c = label_data.columns[0]
    else:
        c = str(som_dim)+'_'+str(n_clust)
    df = label_data.loc[pd.IndexSlice[:,str(year)],[c]].reset_index()
    df.date = df.date.dt.date
    
    fig = df.iplot(kind='heatmap', title='Daily cluster labels for profiles in '+str(year)+
                   ' (cluster ='+c+')', x='date', y='ProfileID', z=c, colorscale='spectral', asFigure=True)

    fig['layout']['yaxis'].update(dict(type='category',title='ProfileID'))
    fig['layout']['xaxis'].update(dict(title='Date'))
    for i, trace in enumerate(fig['data']):
        hovertext = list()
        for j in range(len(trace['x'])):
            hovertext.append('date: {}<br />cluster label: {}<br />ProfileID: {}<br />'.format(trace['x'][j], trace['z'][j]+1, trace['y'][j]))
        trace['text'] = hovertext
        trace['hoverinfo']='text'
    
    return po.iplot(fig)

In [ ]:
plotClusterLabels(sanorm_labels, 2010)

In [ ]:
def display_cmap(cmap): #Display  a colormap cmap
    plt.imshow(np.linspace(0, 100, 256)[None, :],  aspect=25, interpolation='nearest', cmap=cmap) 
    plt.axis('off')
    
def colormap_to_colorscale(cmap):
    #function that transforms a matplotlib colormap to a Plotly colorscale
    return [ [k*0.1, colors.rgb2hex(cmap(k*0.1))] for k in range(11)]

def colorscale_from_list(alist, name): 
    # Defines a colormap, and the corresponding Plotly colorscale from the list alist
    # alist=the list of basic colors
    # name is the name of the corresponding matplotlib colormap
    
    cmap = LinearSegmentedColormap.from_list(name, alist)
#    display_cmap(cmap)
    colorscale=colormap_to_colorscale(cmap)
    return cmap, colorscale

def normalize(x,a,b): #maps  the interval [a,b]  to [0,1]
    if a>=b:
        raise ValueError('(a,b) is not an interval')
    return float(x-a)/(b-a)

def asymmetric_colorscale(data,  div_cmap, ref_point=0.0, step=0.05):
    #data: data can be a DataFrame, list of equal length lists, np.array, np.ma.array
    #div_cmap is the symmetric diverging matplotlib or custom colormap
    #ref_point:  reference point
    #step:  is step size for t in [0,1] to evaluate the colormap at t
   
    if isinstance(data, pd.DataFrame):
        D = data.values
    elif isinstance(data, np.ma.core.MaskedArray):
        D=np.ma.copy(data)
    else:    
        D=np.asarray(data, dtype=np.float) 
    
    dmin=np.nanmin(D)
    dmax=np.nanmax(D)
    if not (dmin < ref_point < dmax):
        raise ValueError('data are not appropriate for a diverging colormap')
        
    if dmax+dmin > 2.0*ref_point:
        left=2*ref_point-dmax
        right=dmax
        
        s=normalize(dmin, left,right)
        refp_norm=normalize(ref_point, left, right)# normalize reference point
        
        T=np.arange(refp_norm, s, -step).tolist()+[s]
        T=T[::-1]+np.arange(refp_norm+step, 1, step).tolist()
        
        
    else: 
        left=dmin
        right=2*ref_point-dmin
        
        s=normalize(dmax, left,right) 
        refp_norm=normalize(ref_point, left, right)
        
        T=np.arange(refp_norm, 0, -step).tolist()+[0]
        T=T[::-1]+np.arange(refp_norm+step, s, step).tolist()+[s]
        
    L=len(T)
    T_norm=[normalize(T[k],T[0],T[-1]) for k in range(L)] #normalize T values  
    return [[T_norm[k], colors.rgb2hex(div_cmap(T[k]))] for k in range(L)]

In [ ]:
def plotClusterSpecificity(data, corr_list, n_clust=None):
    
    print('n_clust options: \n', data.columns.values)
    
    if n_clust is None:
        n_clust = data.columns[0]

    n_corr = len(corr_list)    
    
    #Create dataframes for plot
#    df = data.reset_index()
    
    subplt_titls = ()
    titles = []
    for corr in corr_list:
        title = '"Greater than random" probability of '+corr+' assigned to cluster'
        titles.append((title, None))    
    for t in titles:
        subplt_titls += t
    
    #Initialise plot
    fig = tools.make_subplots(rows=n_corr, cols=2, shared_xaxes=False, print_grid=False, 
                              subplot_titles=subplt_titls)
    #Create colour scale
    smarties = cl.scales['5']['div']['Spectral']
    slatered=['#232c2e', '#ffffff','#c34513']
    label_cmap, label_cs = colorscale_from_list(slatered, 'label_cmap') 
    
    i = 1
    for corr in corr_list:
        function = 'ce.'+corr+'_corr(data, n_clust)'
        rndm_lklhd, lbls2 = eval(function)   

        #Create colorscales
        colorscl= asymmetric_colorscale(lbls2, label_cmap, ref_point=1.0)
#        colorscl=[[0.0, 'rgb(112,138,144)'],[white, 'rgb(255,255,255)'],[1.0, 'rgb(239,138,98)']]

        #Create traces
        heatmap = go.Heatmap(z = lbls2.T.values, x = lbls2.index, y = lbls2.columns, name = corr, 
                          colorscale=colorscl, colorbar=dict(title='likelihood',len=0.9/n_corr, y= 1-i/n_corr+0.05/i, yanchor='bottom'))
        bargraph = lbls2.iplot(kind='bar', colors=smarties, showlegend=False, asFigure=True)

        fig.append_trace(heatmap, i, 1)
        for b in bargraph['data']:
            fig.append_trace(b, i, 2)
        random_likelihood=dict(type='scatter', x=[lbls2.index[0], lbls2.index[-1]], y=[1, 1], 
                                       mode='lines', line=dict(color='black',dash='dash'))
        fig.append_trace(random_likelihood, i, 2)
        
        fig['layout']['yaxis'+str(i*2)].update(title='greater than random Passignment')
        fig['layout']['annotations'].extend([dict(x = lbls2.index[int(len(lbls2.index)*0.5)], y = 1, showarrow=True, yshift=5,
                                              text="random assignment",ax=10, ay=-70, xref='x'+str(i*2), yref='y'+str(i*2))])
        
        i += 1

    #Update layout
    fig['layout'].update(title='Temporal specificity of k clusters', height=n_corr*400, hovermode = "closest", showlegend=False) 

    po.iplot(fig)

In [ ]:
rndm_lklhd, lbls2 = ce.weekday_corr(sanorm_labels, '0_17')
lbls2.iplot(kind='heatmap',colorscale='RdGy')

## Visualise Clusters and Label Assignment

In [ ]:
#plotClusterCentroids(sanorm_clusters)
plotClusterSpecificity(sanorm_labels, corr_list=['daytype','weekday','month','season','year'])

In [ ]:
plotClusterCentroids(best_clusters)
#plotClusterSpecificity(norm_labels, corr_list=['daytype','weekday','month','season','year'])

In [ ]:
plotClusterCentroids(demin_clusters)
plotClusterSpecificity(demin_labels, corr_list=['daytype','weekday','month','season','year'])

In [ ]:
plotClusterCentroids(zerone_clusters)
plotClusterSpecificity(zerone_labels, corr_list=['daytype','weekday','month','season','year'])

### Visualising Daily Demand Assignment

In [ ]:
int100_likelihood, q100_likelihood = ce.demandCorr('exp2_norm_kmeans', '0_47')

In [ ]:
#Equally spaced daily demand intervals
i = int100_likelihood.stack().reset_index()
i.columns = ['int100_bins', 'cluster', 'values']
fig = i.iplot(kind='heatmap', x = 'int100_bins', y='cluster', z='values', colorscale='Reds', 
              title= 'Heatmap of relative likelihood of Cluster k being used in consumption bin', asFigure=True)
fig['layout']['xaxis'].update(dict(title = 'total daily demand bins (Amps)', 
                                   tickmode='array', tickvals=list(range(0,100,10)), ticktext = list(range(0,1000,100))))
fig['layout']['yaxis'].update(dict(title='Cluster k'))
po.iplot(fig)

#Equally sized daily demand intervals (quantiles)
rel_q100 = q100_likelihood.drop(columns='Cluster 33')/0.01

slatered=['#232c2e', '#ffffff','#c34513']
label_cmap, label_cs = colorscale_from_list(slatered, 'label_cmap') 
colorscl= asymmetric_colorscale(rel_q100, label_cmap, ref_point=1.0)

heatmap = go.Heatmap(z = rel_q100.T.values, x = rel_q100.index, y = rel_q100.columns, name = 'corr', 
                          colorscale=colorscl)
layout = go.Layout(
        title= 'Heatmap of relative likelihood of Cluster k being used in consumption quantile',
        xaxis=dict(title = 'total daily demand quantiles (Amps) - log scale', type='log'),
        yaxis=dict(title ='Cluster k'))
fig = {'data':[heatmap], 'layout':layout }
po.iplot(fig)